In [1]:
import setup
setup.init_django(project_name='conf')

In [2]:
import helpers

In [3]:
from bs4 import BeautifulSoup

In [4]:
url = "https://www.amazon.com/PlayStation%C2%AE5-console-slim-PlayStation-5/dp/B0CL61F39H/ref=sr_1_1?crid=30GAK7NM13LAY&dib=eyJ2IjoiMSJ9.F-c6KBJm1tKH0t43iBScfCfEIEFn9fme4tekt5SU7EijIEwRh738VBI7SBXhXPO-5rgEWvxbZTcf8nCJnstwLKn7991qn6xL-T81fzxIuBXSbDxlbzAbuNTXEIILAFc9lg2Xr7HRncSoyhDqnlguDwVApeTf_RZic9BNKs7dqJSpsvAjEMNR3OiKYWy1y8ZzXMZmJ1Yz7gFZHkxiyW7p9XlVt8i9Q-DolMmjCnL02BU.hovw4FtCiftxgW1SnWH1VUZru8X9Qxg_e76HflKud44&dib_tag=se&keywords=playstation%2B5&qid=1730985769&sprefix=play%2Caps%2C636&sr=8-1&th=1"

In [ ]:
url = "https://www.amazon.com/PlayStation%C2%AE5-Digital-slim-PlayStation-5/dp/B0CL5KNB9M/ref=sr_1_3?crid=B7GWFZXZ2OED&dib=eyJ2IjoiMSJ9.kgp6If9Ie8zGHwBo-0htBLyQbbKjs5VuqpcJV5opH4mRqQT9y1GDUhgEGC4Ze5c7iOpklu5U3l_vF3hGTmGfZf8jvVY7cSGvtmhRbSth2-wUchP4cPB4bCopxZnBPpqLbX4wU-JZkepl_i4fGdXQJXUMLc256FqdCdlbjr6ZMyFHhWIJq2G38fcfQx3z9RS1e48jNXaYXv1rWtJ3Y30-OZP-ckGz15zF5vR6k6z6G6c.HDrf64xu0Nz7DYLZvUdGglWizRZpAXBxsxbsgsW26Tc&dib_tag=se&keywords=ps5&qid=1709675943&sprefix=ps5%2Caps%2C190&sr=8-3"

In [5]:
url

'https://www.amazon.com/PlayStation%C2%AE5-console-slim-PlayStation-5/dp/B0CL61F39H/ref=sr_1_1?crid=30GAK7NM13LAY&dib=eyJ2IjoiMSJ9.F-c6KBJm1tKH0t43iBScfCfEIEFn9fme4tekt5SU7EijIEwRh738VBI7SBXhXPO-5rgEWvxbZTcf8nCJnstwLKn7991qn6xL-T81fzxIuBXSbDxlbzAbuNTXEIILAFc9lg2Xr7HRncSoyhDqnlguDwVApeTf_RZic9BNKs7dqJSpsvAjEMNR3OiKYWy1y8ZzXMZmJ1Yz7gFZHkxiyW7p9XlVt8i9Q-DolMmjCnL02BU.hovw4FtCiftxgW1SnWH1VUZru8X9Qxg_e76HflKud44&dib_tag=se&keywords=playstation%2B5&qid=1730985769&sprefix=play%2Caps%2C636&sr=8-1&th=1'

In [8]:
html = helpers.scrape(url=url, solve_captcha=False)
data = helpers.extract_amazon_product_data(html)

Connecting to Scraping Browser...
Connected! Navigating to https://www.amazon.com/PlayStation%C2%AE5-console-slim-PlayStation-5/dp/B0CL61F39H/ref=sr_1_1
Navigated! Scraping page content...


In [9]:
data

{'asin': 'B0CL61F39H',
 'title': 'PlayStation®5 console (slim)',
 'price_raw': '499.',
 'price_text': '499.',
 'price': 499.0,
 'metadata': [{'ASIN': 'B0CL61F39H'},
  {'Release date': 'December 10, 2023'},
  {'Customer Reviews': '4.7  4.7 out of 5 stars    \n    8,554 ratings   \n\n\n 4.7 out of 5 stars'},
  {'Best Sellers Rank': '#16 in Video Games (See Top 100 in Video Games)  #1 in PlayStation 5 Consoles'},
  {'Product Dimensions': '14 x 17 x 7 inches; 10.6 Pounds'},
  {'Type of item': 'Video Game'},
  {'Item model number': 'CFI-2015'},
  {'Item Weight': '10.6 pounds'},
  {'Manufacturer': 'Sony'},
  {'Country of Origin': 'China'},
  {'Batteries': '1 Lithium Ion batteries required. (included)'},
  {'Date First Available': 'December 10, 2023'}],
 'description': ' \n The PS5 console unleashes new gaming possibilities that you never anticipated. Experience lightning fast loading with an ultra-high speed SSD, deeper immersion with support for haptic feedback, adaptive triggers, and 3D Au

In [ ]:
def find_product_table_data(html):
    soup = BeautifulSoup(html)
    product_data = soup.find('div', id='prodDetails')
    if product_data is None:
        return []
    table = product_data.find('table')
    columns = [f"{x.text}".strip() for x in table.find_all('th')]
    table_data=[]
    for i, row in enumerate(table.find_all('tr')):  # [1:] to skip the header row
        # Get all cells in the row
        cells = row.find_all('td')
        # Create a dictionary for the current row, mapping header to cell data
        row_data = {columns[i]: f'{cell.text}'.strip() for cell in cells}
        # Add the dictionary to your list
        table_data.append(row_data)
    return table_data

def find_product_rating(html):
    soup = BeautifulSoup(html)
    average_rating = soup.find(id='averageCustomerReviews').find_all("span", class_='a-size-base')[0].text.strip()
    average_rating = "".join([x for x in f"{average_rating}".strip() if x.isdigit() or x == '.'])
    average_rating = float(average_rating)
    rating_data = soup.find(id='acrCustomerReviewText').text
    rating_count = int(''.join([x for x in rating_data if x.isdigit()]))
    rating_count
    return {
        'average': average_rating,
        'count': rating_count,
    }

def extract_amazon_product_data(html):
    soup = BeautifulSoup(html)
    productTitle = soup.find('span', id='productTitle')
    productTitleText = f"{productTitle.text}".strip()
    productPrice = soup.find_all('span', class_='a-price-whole')[0]
    productPrice = f"{productPrice.text}".strip()
    productPriceText = "".join([x for x in productPrice if x.isdigit() or x == '.'])
    productPriceNum = float(productPriceText)
    try:
        productDescription = soup.find('div', id='productDescription').text
    except:
        productDescription = ''
    featureBullets = soup.find('div', id='feature-bullets').text
    return {
        'title': productTitleText,
        'price_raw': productPrice,
        'price_text': productPriceText,
        'price': productPriceNum,
        'metadata': find_product_table_data(html),
        'description': productDescription,
        'feature_bullets': featureBullets,
        'rating': find_product_rating(html)
    }

In [ ]:
html = helpers.scrape(url=url, solve_captcha=False)

In [ ]:
data = extract_amazon_product_data(html)
data